# 미들웨어 & 가드레일
이 파트에서는 LangChain 에이전트의 실행 흐름을 제어하고 안전성을 보장하는 미들웨어와 가드레일을 다룹니다. 
미들웨어는 에이전트 실행 파이프라인에 끼워넣는 훅 함수이며, 가드레일은 에이전트의 입출력을 검증하고 제어하는 안전 장치입니다.

* 미들웨어 이해 
    에이전트 실행 파이프라인의 각 단계에 개입하는 훅 시스템
* 내장 미들웨어 활용 
    요약, 재시도, 호출 제한 등 프로덕션 레디 미들웨어
* 커스텀 미들웨어 작성 
    데코레이터/클래스 기반 구현으로 비즈니스 로직 통합
* 가드레일 적용 
    PII 보호, Human-in-the-Loop으로 안전한 에이전트 구축

## 주요 내용
|섹션|내용|핵심 기술|
|--|--|--|
|미들웨어 개요|훅 타입, 실행 순서|Node-style, Wrap-style|
|내장 미들웨어|프로덕션 레디 미들웨어|SummarizationMiddleware, ModelCallLimitMiddleware|
|커스텀 미들웨어|직접 미들웨어 작성|@before_model, @wrap_model_call|
|가드레일 개요|안전 장치 개념|결정적/모델 기반 가드레일|
|PII 탐지|개인정보 보호|PIIMiddleware, 4가지 전략|
|Human-in-the-Loop|사람 승인 워크플로우|승인/편집/거부|

In [4]:
# 미들웨어 vs 가드레일
# 미들웨어는 에이전트 실행의 각 단계(모델 호출 전/후, 도구 호출 전/후)에 개입하여 로깅, 검증, 변환 등의 작업을 수행합니다

from langchain.agents import create_agent
from langchain.agents.middleware import before_model
from langchain.tools import tool, ToolRuntime

@tool
def search_tool(query: str, runtime: ToolRuntime) -> str:
    """웹 검색 도구"""
    return f"검색 결과: {query}"


@before_model
def log_input(state, runtime):
    print(f"모델 입력: {state['messages'][-1].content}")
    return None

agent = create_agent(
    model="openai:gpt-4o",
    tools=[search_tool],
    middleware=[log_input]
)

result = agent.invoke({"messages": [{"content": "오늘 날씨 어때?", "role": "user"}]})
result["messages"][-1].content


모델 입력: 오늘 날씨 어때?
모델 입력: 검색 결과: 오늘 날씨 서울


'오늘 서울의 날씨를 알려드릴게요.\n\n- 현재 기온: 약 18도\n- 날씨: 맑음\n- 최고기온: 약 21도\n- 최저기온: 약 14도\n- 바람: 북동풍, 약간의 바람\n\n특별한 기상 경보는 없는 것으로 보입니다. 계획하신 외출에 참고하세요!'

* 가드레일은 미들웨어의 특수한 형태로, 보안과 안전에 초점을 맞춥니다

- 입력 가드레일
    악의적 프롬프트, PII 유출 방지
- 출력 가드레일
    부적절한 응답, 할루시네이션 차단
- 도구 호출 가드레일
    위험한 작업 승인 요청 (Human-in-the-Loop)

## 미들웨어 개요
미들웨어(Middleware)는 LangChain 에이전트의 실행 파이프라인에 끼워넣을 수 있는 훅(hook) 함수입니다. 
에이전트가 모델을 호출하거나 도구를 실행하는 과정에서 미들웨어를 통해 다음과 같은 작업을 수행할 수 있습니다

- 동작 추적
    로깅, 분석, 디버깅
- 프롬프트 변환
    도구 선택, 출력 포맷 조정
- 재시도 및 폴백
    실패 시 재시도하거나 대체 모델 사용
- 가드레일 적용
    호출 제한, PII 탐지, 콘텐츠 필터링

미들웨어를 사용하면 에이전트의 핵심 로직을 수정하지 않고도 횡단 관심사(cross-cutting concerns)를 처리할 수 있습니다.

### 에이전트 루프와 미들웨어
에이전트는 다음과 같은 기본 루프를 실행합니다:

1. 모델 호출 : LLM을 호출하여 다음 행동 결정
2. 도구 실행 : 모델이 선택한 도구를 실행
3. 종료 판단 : 더 이상 도구 호출이 없으면 종료

미들웨어는 이 루프의 각 단계 전후에 실행되는 훅을 제공합니다:

* 에이전트 실행 시작
    before_agent()

* 모델 호출 전
    before_model()

* 모델 API 호출
    after_model()

* 도구 실행 전
    before_tool()
    
* 도구 실행
    after_tool()

* 에이전트 실행 종료
    after_agent()


### 미들웨어 훅 타입
미들웨어는 두 가지 스타일의 훅을 제공합니다.

* Node-style 훅
  특정 실행 지점에서 순차적으로 실행되는 훅입니다. 로깅, 검증, 상태 업데이트에 주로 사용됩니다.

  |훅|실행 시점|실행 횟수|
  |--|--|--|
  |before_agent|에이전트 시작 전|호출당 1회|
  |before_model|모델 호출 전|모델 호출마다|
  |after_model|모델 응답 후|모델 호출마다|
  |after_agent|에이전트 종료 후|호출당 1회|

* Wrap-style 훅
  실행을 감싸서(wrap) 제어하는 훅입니다. 
  재시도, 캐싱, 변환 작업에 주로 사용됩니다. 핸들러를 0회(단락), 1회(정상), 또는 여러 번(재시도) 호출할 수 있습니다.

  |훅|감싸는 대상|용도|
  |--|--|--|
  |wrap_model_call|모델 호출|재시도, 캐싱, 폴백|
  |wrap_tool_call|도구 호출|재시도, 검증, 캐싱|

In [ ]:
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_agent, after_agent
from langchain_core.messages import HumanMessage
from langgraph.runtime import Runtime
import time

# 시작 시간 저장을 위한 전역 변수
start_times = {}

@before_agent
def start_timer(state: AgentState, runtime: Runtime):
    """에이전트 시작 시간 기록"""
    thread_id = runtime.config.get("configurable", {}).get("thread_id", "default")
    start_times[thread_id] = time.time()
    return None

@after_agent
def end_timer(state: AgentState, runtime: Runtime):
    """에이전트 종료 시간 기록 및 출력"""
    thread_id = runtime.config.get("configurable", {}).get("thread_id", "default")
    elapsed = time.time() - start_times.get(thread_id, time.time())
    print(f"에이전트 실행 시간: {elapsed:.2f}초")
    return None

# 에이전트 생성
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search_tool, calculator_tool],
    middleware=[start_timer, end_timer]
)

# 실행
response = agent.invoke(
    {"messages": [HumanMessage(content="서울 인구는?")]},
    config={"configurable": {"thread_id": "thread_1"}}
)
# 출력: 에이전트 실행 시간: 2.34초


In [ ]:
from langchain.agents import AgentState
from langchain.agents.middleware import wrap_model_call
from langgraph.runtime import Runtime

@wrap_model_call
def retry_on_error(state: AgentState, runtime: Runtime, call_next):
    """모델 호출 실패 시 최대 3회 재시도"""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            # 실제 모델 호출 실행
            return call_next()
        except Exception as e:
            print(f"시도 {attempt + 1} 실패: {e}")
            if attempt == max_retries - 1:
                raise
    return None

# 에이전트에 적용
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search_tool],
    middleware=[retry_on_error]
)

### 데코레이터 파라미터
* state_schema
* can_jump_to

In [ ]:
# state_schema 예제
# 커스텀 상태 스키마를 지정합니다. 
# 미들웨어가 특정 상태 필드에 접근해야 할 때 사용합니다

from typing import TypedDict
from langchain.agents import AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime

class CustomState(AgentState):
    """커스텀 상태 스키마"""
    user_id: str
    request_count: int

@before_model(state_schema=CustomState)
def check_user_limit(state: CustomState, runtime: Runtime):
    """사용자별 요청 횟수 제한"""
    if state.get("request_count", 0) >= 10:
        raise Exception(f"사용자 {state['user_id']}의 요청 한도 초과")
    return None


In [ ]:
# can_jump_to 예제
# 미들웨어가 점프할 수 있는 노드를 지정합니다. 조기 종료나 특정 노드로 이동할 때 사용합니다:

from langchain.agents import AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime

@before_model(can_jump_to=["end"])
def check_budget(state: AgentState, runtime: Runtime):
    """예산 초과 시 에이전트 종료"""
    total_tokens = runtime.context.get("total_tokens", 0)
    if total_tokens > 10000:
        # 'end' 노드로 점프하여 에이전트 종료
        return {"jump_to": "end"}
    return None


### 미들웨어 실행 순서

#### Before 훅: 순서대로 실행
before_* 훅은 미들웨어 리스트 순서대로 실행됩니다:

middleware = [middleware1, middleware2, middleware3]

실행 순서:
    1. middleware1.before_agent()
    2. middleware2.before_agent()
    3. middleware3.before_agent()
    4. middleware1.before_model()
    5. middleware2.before_model()
    6. middleware3.before_model()

#### After 훅: 역순으로 실행
after_* 훅은 미들웨어 리스트의 역순으로 실행됩니다:

모델 호출 후:
    1. middleware3.after_model()
    2. middleware2.after_model()
    3. middleware1.after_model()
에이전트 종료 후:
    1. middleware3.after_agent()
    2. middleware2.after_agent()
    3. middleware1.after_agent()

#### Wrap 훅: 중첩 실행
wrap_* 훅은 함수 호출처럼 중첩됩니다:

실행 순서:
middleware1.wrap_model_call() 진입
  → middleware2.wrap_model_call() 진입
    → middleware3.wrap_model_call() 진입
      → 실제 모델 호출
    ← middleware3.wrap_model_call() 종료
  ← middleware2.wrap_model_call() 종료
← middleware1.wrap_model_call() 종료

### 미들웨어 적용 방법
* 데코레이터 방식
    간단한 단일 훅 미들웨어는 데코레이터로 구현할 수 있습니다:
* 클래스 방식
    복잡한 설정이나 여러 훅이 필요한 경우 클래스로 구현합니다:
* 내장 미들웨어 사용
    LangChain은 다양한 내장 미들웨어를 제공합니다:

In [ ]:
# 데코레이터 방식
from langchain.agents import AgentState
from langchain.agents.middleware import before_model, after_model
from langgraph.runtime import Runtime

@before_model
def log_input(state: AgentState, runtime: Runtime):
    """모델 입력 로깅"""
    print(f"모델 입력: {state['messages'][-1].content}")
    return None

@after_model
def log_output(state: AgentState, runtime: Runtime):
    """모델 출력 로깅"""
    print(f"모델 출력: {state['messages'][-1].content}")
    return None

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search_tool],
    middleware=[log_input, log_output]
)

In [ ]:
# 클래스 방식
from langchain.agents import AgentState
from langchain.agents.middleware import Middleware
from langgraph.runtime import Runtime

class TokenCounterMiddleware(Middleware):
    """토큰 사용량 추적 미들웨어"""

    def __init__(self):
        self.total_tokens = 0

    def before_agent(self, state: AgentState, runtime: Runtime):
        """초기화"""
        self.total_tokens = 0
        return None

    def after_model(self, state: AgentState, runtime: Runtime):
        """모델 호출 후 토큰 카운트"""
        # runtime.context에서 사용량 정보 접근
        usage = runtime.context.get("usage")
        if usage:
            self.total_tokens += usage.get("total_tokens", 0)
        return None

    def after_agent(self, state: AgentState, runtime: Runtime):
        """총 토큰 출력"""
        print(f"총 사용 토큰: {self.total_tokens}")
        return None

# 인스턴스 생성 및 적용
token_counter = TokenCounterMiddleware()
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search_tool],
    middleware=[token_counter]
)


In [7]:
# 내장 미들웨어 사용
from langchain.agents.middleware import (
    ModelCallLimitMiddleware,
    ToolRetryMiddleware,
    PIIMiddleware
)
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search_tool],
    middleware=[
        # 모델 호출 횟수 제한 (최대 5회)
        ModelCallLimitMiddleware(run_limit=5),

        # 도구 실패 시 재시도 (최대 3회)
        ToolRetryMiddleware(max_retries=3),

        # 이메일 주소 자동 마스킹
        PIIMiddleware(
            "email",
            strategy="mask"
        )
    ]
)

response = agent.invoke({
    "messages": [HumanMessage(content="내 이메일 user@example.com로 알림 보내줘")]
})
# 이메일이 자동으로 마스킹됨: "u***@example.com"
response["messages"][-1].content


'죄송하지만, 저는 이메일을 보낼 수 있는 기능이 없습니다. 다른 방법으로 도와드릴 수 있는 것이 있나요?'

### 미들웨어 활용 사례

In [ ]:
# 1. 비용 모니터링
from langchain.agents import AgentState
from langchain.agents.middleware import Middleware
from langgraph.runtime import Runtime

class CostTrackerMiddleware(Middleware):
    """API 비용 추적"""

    def __init__(self, cost_per_1k_tokens=0.002):
        self.cost_per_1k = cost_per_1k_tokens
        self.total_cost = 0.0

    def after_model(self, state: AgentState, runtime: Runtime):
        usage = runtime.context.get("usage")
        if usage:
            tokens = usage.get("total_tokens", 0)
            cost = (tokens / 1000) * self.cost_per_1k
            self.total_cost += cost
        return None

    def after_agent(self, state: AgentState, runtime: Runtime):
        print(f"총 비용: ${self.total_cost:.4f}")
        return None


In [ ]:
# 2. 실행 취소 조건
from langchain.agents import AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime

@before_model(can_jump_to=["end"])
def check_token_limit(state: AgentState, runtime: Runtime):
    """토큰 한도 초과 시 조기 종료"""
    total_tokens = runtime.context.get("total_tokens", 0)
    if total_tokens > 10000:
        # 'end' 노드로 점프하여 에이전트 종료
        return {"jump_to": "end"}
    return None


In [ ]:
# 3. 동적 프롬프트 조정
from langchain.agents import AgentState
from langchain.agents.middleware import dynamic_prompt
from langgraph.runtime import Runtime

@dynamic_prompt
def add_user_context(state: AgentState, runtime: Runtime) -> str:
    """사용자 컨텍스트를 포함한 동적 시스템 프롬프트 생성"""
    user_id = runtime.context.get("user_id")
    user_prefs = get_user_preferences(user_id)

    return f"""당신은 친절한 AI 어시스턴트입니다.

사용자 선호도:
- 언어: {user_prefs.get('language', 'ko')}
- 전문성: {user_prefs.get('expertise', 'beginner')}
"""

def get_user_preferences(user_id):
    return {"language": "ko", "expertise": "intermediate"}


### 내장 미들웨어
LangChain은 프로덕션 환경에서 바로 사용할 수 있는 10가지 내장 미들웨어를 제공

#### 미들웨어 분류
|카테고리|미들웨어|용도|상세|
|--|--|--|--|
|컨텍스트 관리|SummarizationMiddleware|긴 대화 자동 요약|4-2-1|
| |ContextEditingMiddleware|오래된 도구 결과 제거|4-2-1|
|호출 제한|ModelCallLimitMiddleware|모델 API 호출 횟수 제한|4-2-2|
| |ToolCallLimitMiddleware|도구 호출 횟수 제한|4-2-2|
|복원력|ModelFallbackMiddleware|모델 실패 시 대체 모델|4-2-3|
| |ModelRetryMiddleware|모델 호출 재시도|4-2-3|
| |ToolRetryMiddleware|도구 실행 재시도|4-2-3|
|도구 최적화|LLMToolSelectorMiddleware|관련 도구만 선택|4-2-4|
|가드레일|PIIMiddleware|개인정보 탐지 및 마스킹|4-5|
| |HumanInTheLoopMiddleware|사람 승인 후 실행|4-6|

#### 컨텍스트 관리 미들웨어
컨텍스트 관리 미들웨어는 긴 대화에서 토큰 한도를 효율적으로 관리하는 기능을 제공합니다. 
대화가 길어지면 토큰 한도에 도달하거나 비용이 증가하므로, 이를 자동으로 관리하는 미들웨어가 필요합니다.

##### Summarization (대화 요약)
긴 대화 내역이 토큰 한도에 근접하면 자동으로 이전 메시지를 요약하여 컨텍스트를 압축합니다.

주요 용도
* 토큰 한도를 초과하는 장시간 대화
* 대화 히스토리가 많은 멀티턴 다이얼로그
* 전체 컨텍스트 보존이 중요한 애플리케이션

설정 옵션
|옵션|타입|설명|
|--|--|--|
|model|str | BaseChatModel|요약에 사용할 모델 (예: 'openai:gpt-4o-mini')|
|trigger|dict|요약 트리거 조건 (tokens, messages, fraction)|
|keep|dict|요약 후 보존할 컨텍스트 크기|